In [3]:
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

ingredientes_totales_path = "ingredientes_unicos_totales.xlsx"
dataset_v5_path = "dataset_v5.xlsx"

ingredientes_totales_df = pd.read_excel(ingredientes_totales_path)
dataset_v5_df = pd.read_excel(dataset_v5_path)
ingredientes_unicos = ingredientes_totales_df['name'].unique()

# Function to clear ingredient names.
def limpiar_ingredientes(ingrediente):
    ingrediente = ingrediente.strip("[]'\"")
    ingredientes_limpios = [ing.strip("'\" ") for ing in ingrediente.split(',')]
    ingredientes_limpios = [re.split(r'/\d', ing)[0] for ing in ingredientes_limpios]
    return ','.join(ingredientes_limpios)

# Function to obtain the quantity of an ingredient
def obtener_cantidad(ingrediente):
    cantidad_match = re.search(r'/(\d+)', ingrediente)
    return int(cantidad_match.group(1)) if cantidad_match else 0

# Create a dictionary of ingredients to indexes
ingredientes_dict = {ingrediente: idx for idx, ingrediente in enumerate(ingredientes_unicos)}

n_platos = dataset_v5_df['N° Food dish'].nunique()
n_ingredientes = len(ingredientes_unicos)
input_matrix = pd.DataFrame(0, index=range(n_platos), columns=ingredientes_unicos)

grouped_dishes = dataset_v5_df.groupby('N° Food dish')['F.D. List(Ingredient/Portion)'].apply(lambda x: ','.join(x)).reset_index()
grouped_dishes['ingredientes_limpios'] = grouped_dishes['F.D. List(Ingredient/Portion)'].apply(lambda x: limpiar_ingredientes(x))
grouped_dishes['ingredientes_cantidades'] = grouped_dishes['F.D. List(Ingredient/Portion)'].apply(lambda x: [obtener_cantidad(ing) for ing in x.split(',')])

# Update the input array with quantities
for idx, row in grouped_dishes.iterrows():
    ingredientes_plato = row['ingredientes_limpios'].split(',')
    cantidades_plato = row['ingredientes_cantidades']
    for ingrediente, cantidad in zip(ingredientes_plato, cantidades_plato):
        nombre_ingrediente = re.split(r'/\d', ingrediente)[0].strip()
        if nombre_ingrediente in ingredientes_dict:
            input_matrix.at[idx, nombre_ingrediente] = cantidad

# Data normalization
max_value = input_matrix.values.max()
input_matrix = input_matrix / max_value

# Definition of the autoencoder model
input_dim = input_matrix.shape[1]
encoding_dim = 64

input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(412, activation='relu')(input_layer)

decoded = layers.Dense(206, activation='relu')(encoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = models.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

# Training the autoencoder with the normalized input matrix
X_train = input_matrix.values

autoencoder.fit(X_train, X_train, epochs=500, batch_size=256, shuffle=True)

# Create the encoder model
encoder = models.Model(input_layer, encoded)
encoded_dishes = encoder.predict(X_train)
similarities = cosine_similarity(encoded_dishes)

# Save the similarity matrix
np.save('modelos/model_autoencoder/similarities.npy', similarities)


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 103)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 412)            │        42,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 206)            │        85,078 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 103)            │        21,321 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 149,247 (583.00 KB)

 Trainable params: 149,247 (583.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.6923
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6875
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.6824
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6767
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.6701
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.6626
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6538
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.6436
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.6318
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6183
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6029
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.5854
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.5657
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.5437
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.5193
Epoch 16/500
1/1 ━━━━

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000241D253DEE0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [37]:
def evaluar_recomendaciones(plato_index, top_n=5):
    similar_plates = similarities[plato_index].argsort()[::-1][1:top_n+1]
    
    print(f"Plato {plato_index}:")
    for idx in similar_plates:
        print(f"Plato recomendado: {idx}, Similitud: {similarities[plato_index][idx]:.2f}")

evaluar_recomendaciones(0, 50)

Plato 0:
Plato recomendado: 6, Similitud: 0.70
Plato recomendado: 23, Similitud: 0.68
Plato recomendado: 9, Similitud: 0.68
Plato recomendado: 27, Similitud: 0.67
Plato recomendado: 21, Similitud: 0.66
Plato recomendado: 7, Similitud: 0.66
Plato recomendado: 40, Similitud: 0.65
Plato recomendado: 29, Similitud: 0.65
Plato recomendado: 30, Similitud: 0.64
Plato recomendado: 38, Similitud: 0.63
Plato recomendado: 39, Similitud: 0.62
Plato recomendado: 45, Similitud: 0.62
Plato recomendado: 2, Similitud: 0.61
Plato recomendado: 13, Similitud: 0.61
Plato recomendado: 46, Similitud: 0.58
Plato recomendado: 5, Similitud: 0.58
Plato recomendado: 32, Similitud: 0.58
Plato recomendado: 31, Similitud: 0.58
Plato recomendado: 22, Similitud: 0.57
Plato recomendado: 44, Similitud: 0.57
Plato recomendado: 48, Similitud: 0.56
Plato recomendado: 26, Similitud: 0.56
Plato recomendado: 11, Similitud: 0.55
Plato recomendado: 36, Similitud: 0.55
Plato recomendado: 19, Similitud: 0.55
Plato recomendado: 10